In [1]:
import torch
from torch import nn
import torchvision.utils as vutils
from torch.optim import Adam
from torch.utils.data import DataLoader,Dataset
import glob
from torchvision import transforms, datasets
from torchvision.utils import save_image
import os
import random
from PIL import Image
import itertools
import matplotlib.pyplot as plt
from torchvision.models import vgg19, resnet18

In [2]:
class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super(ResidualBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_features, in_features, kernel_size=3, padding=1, bias=False),
            nn.InstanceNorm2d(in_features),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_features, in_features, kernel_size=3, padding=1, bias=False),
            nn.InstanceNorm2d(in_features)
        )

    def forward(self, x):
        return x + self.block(x)

class Generator(nn.Module):
    def __init__(self, input_nc, output_nc, n_residual_blocks=9):
        super(Generator, self).__init__()

        # Initial convolution block
        model = [
            nn.Conv2d(input_nc, 64, kernel_size=7, padding=3, bias=False),
            nn.InstanceNorm2d(64),
            nn.ReLU(inplace=True)
        ]

        # Downsampling
        in_features = 64
        out_features = in_features*2
        for _ in range(2):
            model += [
                nn.Conv2d(in_features, out_features, kernel_size=3, stride=2, padding=1, bias=False),
                nn.InstanceNorm2d(out_features),
                nn.ReLU(inplace=True)
            ]
            in_features = out_features
            out_features = in_features*2

        # Residual blocks
        for _ in range(n_residual_blocks):
            model += [ResidualBlock(in_features)]

        # Upsampling
        out_features = in_features//2
        for _ in range(2):
            model += [
                nn.ConvTranspose2d(in_features, out_features, kernel_size=3, stride=2, padding=1, output_padding=1, bias=False),
                nn.InstanceNorm2d(out_features),
                nn.ReLU(inplace=True)
            ]
            in_features = out_features
            out_features = in_features//2

        # Output layer
        model += [nn.Conv2d(64, output_nc, kernel_size=7, padding=3), nn.Tanh()]

        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)


In [3]:
class Discriminator(nn.Module):
    def __init__(self, input_nc):
        super(Discriminator, self).__init__()

        model = [
            nn.Conv2d(input_nc, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True)
        ]

        model += [
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1, bias=False),
            nn.InstanceNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True)
        ]

        model += [
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1, bias=False),
            nn.InstanceNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True)
        ]

        model += [
            nn.Conv2d(256, 512, kernel_size=4, padding=1, bias=False),
            nn.InstanceNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True)
        ]

        model += [nn.Conv2d(512, 1, kernel_size=4, padding=1)]

        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)

In [4]:
transform = transforms.Compose([
    transforms.Resize(int(256*1.12), Image.BICUBIC),
    transforms.RandomCrop(256),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


In [12]:
class ImageDataset(Dataset):
    def __init__(self, root='data', transforms_=None, unaligned=False, mode="train"):          ## (root = "./datasets/facades", unaligned=True:非对其数据)
        self.transform = transforms_                             ## transform变为tensor数据
        self.unaligned = unaligned

        self.files_A = sorted(glob.glob(os.path.join(root, "%sA" % mode) + "/*.*"))     ## "./datasets/facades/trainA/*.*"
        self.files_B = sorted(glob.glob(os.path.join(root, "%sB" % mode) + "/*.*"))     ## "./datasets/facades/trainB/*.*"

    def __getitem__(self, index):
        image_A = Image.open(self.files_A[index % len(self.files_A)])                   ## 在A中取一张照片

        if self.unaligned:                                                              ## 如果采用非配对数据，在B中随机取一张
            image_B = Image.open(self.files_B[random.randint(0, len(self.files_B) - 1)])
        else:
            image_B = Image.open(self.files_B[index % len(self.files_B)])

        # # 如果是灰度图，把灰度图转换为RGB图
        # if image_A.mode != "RGB":
        #     image_A = to_rgb(image_A)
        # if image_B.mode != "RGB":
        #     image_B = to_rgb(image_B)
        
        # 把RGB图像转换为tensor图, 方便计算，返回字典数据
        item_A = self.transform(image_A)
        item_B = self.transform(image_B)
        return item_A, item_B

    ## 获取A,B数据的长度
    def __len__(self):
        return max(len(self.files_A), len(self.files_B))


In [13]:
train_dataloader = DataLoader(        ## 改成自己存放文件的目录
    ImageDataset("human_dog", transforms_=transform, unaligned=True, mode="train"),  ## "./datasets/facades" , unaligned:设置非对其数据
    batch_size=4,                                                                  ## batch_size = 1
    shuffle=True,
)
test_dataloader = DataLoader(        ## 改成自己存放文件的目录
    ImageDataset("human_dog", transforms_=transform, unaligned=True, mode="test"),  ## "./datasets/facades" , unaligned:设置非对其数据
    batch_size=4,                                                                  ## batch_size = 1
    shuffle=False,
)


In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

G_AB = Generator(input_nc=3, output_nc=3).to(device)
G_BA = Generator(input_nc=3, output_nc=3).to(device)
D_A = Discriminator(input_nc=3).to(device)
D_B = Discriminator(input_nc=3).to(device)

# Optimizers
optimizer_G = Adam(itertools.chain(G_AB.parameters(), G_BA.parameters()), lr=0.0005, betas=(0.5, 0.999))
optimizer_D_A = Adam(D_A.parameters(), lr=0.0005, betas=(0.5, 0.999))
optimizer_D_B = Adam(D_B.parameters(), lr=0.0005, betas=(0.5, 0.999))

# Losses
criterion_GAN = nn.MSELoss()
criterion_cycle = nn.L1Loss()
criterion_identity = nn.L1Loss()


output_dir = './cyclegan_images'
os.makedirs(output_dir, exist_ok=True)

In [23]:
for epoch in range(4):
    for i, (real_A, real_B) in enumerate(train_dataloader):
        real_A = real_A.to(device)
        real_B = real_B.to(device)

        # 训练生成器 G_A 和 G_B
        optimizer_G.zero_grad()
        
        # 对抗性损失
        fake_B = G_AB(real_A)
        pred_fake = D_B(fake_B)
        loss_GAN_A2B = criterion_GAN(pred_fake, torch.ones_like(pred_fake))
        
        fake_A = G_BA(real_B)
        pred_fake = D_A(fake_A)
        loss_GAN_B2A = criterion_GAN(pred_fake, torch.ones_like(pred_fake))
        
        # 循环一致性损失
        recovered_A = G_BA(fake_B)
        loss_cycle_ABA = criterion_cycle(recovered_A, real_A) * 10.0
        
        recovered_B = G_AB(fake_A)
        loss_cycle_BAB = criterion_cycle(recovered_B, real_B) * 10.0

        # 总损失
        loss_G = loss_GAN_A2B + loss_GAN_B2A + loss_cycle_ABA + loss_cycle_BAB
        loss_G.backward()
        optimizer_G.step()

        # 训练判别器 D_A
        optimizer_D_A.zero_grad()

        pred_real = D_A(real_A)
        loss_D_real = criterion_GAN(pred_real, torch.ones_like(pred_real))

        pred_fake = D_A(fake_A.detach())
        loss_D_fake = criterion_GAN(pred_fake, torch.zeros_like(pred_fake))

        # 总损失
        loss_D_A = (loss_D_real + loss_D_fake) * 0.5
        loss_D_A.backward()
        optimizer_D_A.step()

        # 训练判别器 D_B
        optimizer_D_B.zero_grad()

        pred_real = D_B(real_B)
        loss_D_real = criterion_GAN(pred_real, torch.ones_like(pred_real))

        pred_fake = D_B(fake_B.detach())
        loss_D_fake = criterion_GAN(pred_fake, torch.zeros_like(pred_fake))

        # 总损失
        loss_D_B = (loss_D_real + loss_D_fake) * 0.5
        loss_D_B.backward()
        optimizer_D_B.step()
        
        if i % 10 == 0:
            print(f'Epoch: {epoch}, Batch: {i}, Loss_G: {loss_G.item()}, Loss_D_A: {loss_D_A.item()}, Loss_D_B: {loss_D_B.item()}')

    with torch.no_grad():
        # 使用测试集中的数据生成图像
        for i, (real_A, real_B) in enumerate(test_dataloader):
            real_A = real_A.to(device)
            fake_B = G_AB(real_A)
            vutils.save_image(fake_B, f'{output_dir}/fake_B_epoch_{epoch}_batch_{i}.png', normalize=True)
        


Epoch: 0, Batch: 0, Loss_G: 15.61037826538086, Loss_D_A: 1.1615573167800903, Loss_D_B: 0.9101475477218628
Epoch: 0, Batch: 10, Loss_G: 6.970175743103027, Loss_D_A: 0.3666701912879944, Loss_D_B: 0.24359501898288727
Epoch: 0, Batch: 20, Loss_G: 5.965689659118652, Loss_D_A: 0.24649454653263092, Loss_D_B: 0.24288181960582733
Epoch: 0, Batch: 30, Loss_G: 5.182547569274902, Loss_D_A: 0.24978424608707428, Loss_D_B: 0.2856913208961487
Epoch: 0, Batch: 40, Loss_G: 7.472266674041748, Loss_D_A: 0.3758556544780731, Loss_D_B: 0.32502102851867676
Epoch: 0, Batch: 50, Loss_G: 7.623085975646973, Loss_D_A: 0.23080357909202576, Loss_D_B: 0.24449694156646729
Epoch: 0, Batch: 60, Loss_G: 6.9660868644714355, Loss_D_A: 0.26649582386016846, Loss_D_B: 0.23975417017936707
Epoch: 0, Batch: 70, Loss_G: 5.563512325286865, Loss_D_A: 0.24589692056179047, Loss_D_B: 0.2512208819389343
Epoch: 0, Batch: 80, Loss_G: 6.209434986114502, Loss_D_A: 0.30268868803977966, Loss_D_B: 0.2863529324531555


### GAN Inversion

Train the model to convert A to A. Then interpolation can be used to generate intermediate image. 

Using contrastive loss to make the model to learn?

In [5]:
class ImageDataset(Dataset):
    def __init__(self, root='data', transforms_=None, unaligned=False, mode="train"):          ## (root = "./datasets/facades", unaligned=True:非对其数据)
        self.transform = transforms_                             ## transform变为tensor数据
        self.unaligned = unaligned

        self.files_A = sorted(glob.glob(os.path.join(root, 'combined') + "/*.*"))     ## "./datasets/facades/trainA/*.*"
        self.files_B = sorted(glob.glob(os.path.join(root, 'combined') + "/*.*"))     ## "./datasets/facades/trainB/*.*"

    def __getitem__(self, index):
        image_A = Image.open(self.files_A[index % len(self.files_A)])                   ## 在A中取一张照片

        if self.unaligned:                                                              ## 如果采用非配对数据，在B中随机取一张
            image_B = Image.open(self.files_B[random.randint(0, len(self.files_B) - 1)])
        else:
            image_B = Image.open(self.files_B[index % len(self.files_B)])

        # # 如果是灰度图，把灰度图转换为RGB图
        # if image_A.mode != "RGB":
        #     image_A = to_rgb(image_A)
        # if image_B.mode != "RGB":
        #     image_B = to_rgb(image_B)
        
        # 把RGB图像转换为tensor图, 方便计算，返回字典数据
        item_A = self.transform(image_A)
        item_B = self.transform(image_B)
        return item_A, item_B

    ## 获取A,B数据的长度
    def __len__(self):
        return max(len(self.files_A), len(self.files_B))


In [6]:
train_dataloader = DataLoader(        ## 改成自己存放文件的目录
    ImageDataset("human_dog", transforms_=transform, unaligned=True, mode="train"),  ## "./datasets/facades" , unaligned:设置非对其数据
    batch_size=16,                                                                  ## batch_size = 1
    shuffle=True,
)
test_dataloader = DataLoader(        ## 改成自己存放文件的目录
    ImageDataset("human_dog", transforms_=transform, unaligned=True, mode="test"),  ## "./datasets/facades" , unaligned:设置非对其数据
    batch_size=16,                                                                  ## batch_size = 1
    shuffle=False,
)


In [36]:
import torch
import torch.nn as nn

class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_features, in_features, kernel_size=3, padding=1, bias=False)
        self.norm1 = nn.InstanceNorm2d(in_features)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(in_features, in_features, kernel_size=3, padding=1, bias=False)
        self.norm2 = nn.InstanceNorm2d(in_features)

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.norm1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.norm2(out)
        return identity + out

class DownsampleBlock(nn.Module):
    def __init__(self, in_features, out_features):
        super(DownsampleBlock, self).__init__()
        self.conv = nn.Conv2d(in_features, out_features, kernel_size=3, stride=2, padding=1, bias=False)
        self.norm = nn.InstanceNorm2d(out_features)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.norm(x)
        x = self.relu(x)
        return x

class UpsampleBlock(nn.Module):
    def __init__(self, in_features, out_features):
        super(UpsampleBlock, self).__init__()
        self.conv = nn.ConvTranspose2d(in_features, out_features, kernel_size=3, stride=2, padding=1, output_padding=1, bias=False)
        self.norm = nn.InstanceNorm2d(out_features)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.norm(x)
        x = self.relu(x)
        return x

class Generator(nn.Module):
    def __init__(self, input_nc, output_nc, n_residual_blocks=9):
        super(Generator, self).__init__()
        self.initial_conv = nn.Sequential(
            nn.Conv2d(input_nc, 64, kernel_size=7, padding=3, bias=False),
            nn.InstanceNorm2d(64),
            nn.ReLU(inplace=True)
        )
        self.downsample_blocks = nn.ModuleList([
            DownsampleBlock(64, 128),
            DownsampleBlock(128, 256)
        ])
        self.residual_blocks = nn.Sequential(
            *[ResidualBlock(256) for _ in range(n_residual_blocks)]
        )
        self.upsample_blocks = nn.ModuleList([
            UpsampleBlock(256, 128),
            UpsampleBlock(128, 64)
        ])
        self.output_conv = nn.Sequential(
            nn.Conv2d(64, output_nc, kernel_size=7, padding=3),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.initial_conv(x)
        
        for down_block in self.downsample_blocks:
            x = down_block(x)
        x = self.residual_blocks(x)
        for up_block in self.upsample_blocks:
            x = up_block(x)
        x = self.output_conv(x)
        return x


In [37]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

G_AA = Generator(input_nc=4, output_nc=3).to(device)
# G_BA = Generator(input_nc=3, output_nc=3).to(device)
D_A = Discriminator(input_nc=3).to(device)
D_B = Discriminator(input_nc=3).to(device)

# Optimizers
optimizer_G = Adam(G_AA.parameters(), lr=0.0005, betas=(0.5, 0.999))
optimizer_D_A = Adam(D_A.parameters(), lr=0.0005, betas=(0.5, 0.999))
optimizer_D_B = Adam(D_B.parameters(), lr=0.0005, betas=(0.5, 0.999))

# Losses
criterion_GAN = nn.MSELoss()
criterion_cycle = nn.L1Loss()
criterion_identity = nn.L1Loss()


output_dir = './cyclegan_images'
os.makedirs(output_dir, exist_ok=True)

In [38]:
# 加载ResNet模型
resnet = resnet18(pretrained=True)

# 获取全连接层之前的特征提取部分
features = nn.Sequential(*list(resnet.children())[:-1])

# 定义新的全连接层和ReLU激活函数
num_ftrs = resnet.fc.in_features
fc_layer = nn.Linear(num_ftrs, 256)
relu = nn.ReLU(inplace=True)

# 定义模型结构
class CustomResNet(nn.Module):
    def __init__(self, features, fc_layer, relu, num_classes=1000):
        super(CustomResNet, self).__init__()
        self.features = features
        self.fc_layer = fc_layer
        self.relu = relu
        self.fc_out = nn.Linear(256, num_classes)  # 输出层

    def forward(self, x):
        x = self.features(x)
        print(x.shape)
        x = x.view(x.size(0), -1)
        print(x.shape)
        x = self.fc_layer(x)
        print(x.shape)
        x = self.relu(x)
        print(x.shape)
        x = self.fc_out(x)
        print(x.shape)
        return x


d:\Anaconda\envs\assignment2\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Anaconda\envs\assignment2\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [39]:
# resnet = resnet18(pretrained=True)
# num_ftrs = resnet.fc.in_features
# resnet.fc = nn.Linear(num_ftrs, 256)
# resnet.fc = nn.Linear(32168, 65536)
resnet = torch.load("models\\animal_rec.pth")
resnet.fc_out = torch.nn.Identity()
resnet.relu = torch.nn.Identity()
resnet.fc_layer = torch.nn.Identity()
resnet = resnet.to(device)
resnet.eval()
print(resnet)

CustomResNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats

In [40]:


class PerceptualLoss(nn.Module):
    def __init__(self):
        super(PerceptualLoss, self).__init__()
        # self.vgg = vgg19(pretrained=True).features[:21]  # 只使用到第三个池化层之前的部分
        self.resnet = resnet
        self.loss = nn.MSELoss()

        # 冻结VGG参数
        for param in self.resnet.parameters():
            param.requires_grad = False

    def forward(self, generated, target):
        self.resnet = self.resnet.to(device)
        gen_features = self.resnet(generated)
        target_features = self.resnet(target)

        return self.loss(gen_features, target_features)

# 实例化感知损失
perceptual_loss = PerceptualLoss()


In [41]:
resnet1 = torch.load("models\\animal_rec.pth")
resnet1 = resnet1.to(device)
resnet1.fc_out = torch.nn.Identity()
resnet1.eval()
print(resnet1)

CustomResNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats

In [49]:
for epoch in range(1):
    for i, (real_A, real_B) in enumerate(train_dataloader):
        real_A = real_A.to(device)
        real_B = real_B.to(device)

        # 训练生成器 G_A 和 G_B
        optimizer_G.zero_grad()
        
        # 对抗性损失
        real_A_feature = resnet1(real_A)
        # print(real_A_feature.shape)

        real_A_feature = real_A_feature.unsqueeze(1).unsqueeze(1).expand(-1,-1,256,256)
        real_A_feature = real_A_feature.view(-1,1,256,256)
        # print(real_A_feature.shape)
        real_A_with_feature = torch.cat([real_A,real_A_feature],dim=1)
        # print(real_A_with_feature.shape)
        fake_A = G_AA(real_A_with_feature)
        
        pred_fake = D_A(fake_A)
        loss_GAN_A2B = criterion_GAN(pred_fake, torch.ones_like(pred_fake))
        loss_perceptual = perceptual_loss(fake_A,real_A)

        
        # 循环一致性损失
        loss_cycle_BAB = criterion_cycle(fake_A, real_A) * 10.0

        # 总损失
        loss_G = loss_GAN_A2B + loss_cycle_BAB + loss_perceptual
        loss_G.backward()
        optimizer_G.step()

        # 训练判别器 D_A
        optimizer_D_A.zero_grad()

        pred_real = D_A(real_A)
        loss_D_real = criterion_GAN(pred_real, torch.ones_like(pred_real))

        pred_fake = D_A(fake_A.detach())
        loss_D_fake = criterion_GAN(pred_fake, torch.zeros_like(pred_fake))

        # 总损失
        loss_D_A = (loss_D_real + loss_D_fake) * 0.5
        loss_D_A.backward()
        optimizer_D_A.step()
        
        if i % 10 == 0:
            print(f'Epoch: {epoch}, Batch: {i}, Trained samples: {i*16}, Loss_G: {loss_G.item()}, Loss_D_A: {loss_D_A.item()}')

    # with torch.no_grad():
    #     # 使用测试集中的数据生成图像
    #     for i, (real_A, real_B) in enumerate(test_dataloader):
    #         real_A = real_A.to(device)
    #         real_A_feature = resnet(real_A)
    #         real_A_feature = real_A_feature.unsqueeze(1).unsqueeze(1).expand(-1,-1,256,256)
    #         real_A_with_feature = torch.cat([real_A,real_A_feature],dim=1)
    #         fake_A = G_AA(real_A_with_feature)
    #         vutils.save_image(fake_A, f'{output_dir}/fake_B_epoch_{epoch}_batch_{i}.png', normalize=True)
        


C:\Users\Liu Sicheng\AppData\Local\Temp\ipykernel_175176\3968021234.py:31: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\build\aten\src\ATen/core/TensorBody.h:494.)
  print(loss_G.grad)


None
Epoch: 0, Batch: 0, Trained samples: 0, Loss_G: 1.992967963218689, Loss_D_A: 0.35216641426086426
None


KeyboardInterrupt: 

In [43]:
save_model_path='models'
checkpoint_path = os.path.join(save_model_path, "Generator_PerceptualLoss_withfeature.ckpt")
torch.save(G_AA.state_dict(), checkpoint_path)
print("Model saved at %s" % checkpoint_path)

Model saved at models\Generator_PerceptualLoss_withfeature.ckpt


In [44]:
G_AA = Generator(input_nc=4, output_nc=3).to(device)

In [45]:
G_AA.load_state_dict(torch.load("models\Generator_PerceptualLoss_withfeature.ckpt"))
G_AA.eval()

Generator(
  (initial_conv): Sequential(
    (0): Conv2d(4, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
    (1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (2): ReLU(inplace=True)
  )
  (downsample_blocks): ModuleList(
    (0): DownsampleBlock(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (norm): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (relu): ReLU(inplace=True)
    )
    (1): DownsampleBlock(
      (conv): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (norm): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (relu): ReLU(inplace=True)
    )
  )
  (residual_blocks): Sequential(
    (0): ResidualBlock(
      (conv1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (norm1): InstanceNorm2d(256, eps=

In [46]:
def load_image(image_path):
    image = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([
        transforms.Resize((256, 256)),  # 根据你的模型调整尺寸
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = transform(image).unsqueeze(0)  # 添加批次维度
    return image

In [47]:
def predict(model, image1,image2=None):
    # image1 = load_image(image_path1).to(device)
    # image2 = load_image(image_path2).to(device)

    model= model.to(device)
    for i in range(10):
        with torch.no_grad():  # 不计算梯度
            image=image1*(i/10)+image2*(1-i/10)
            output = model(image)
            vutils.save_image(output, f'test4 {i}.png', normalize=True)
    return output

In [48]:
image1 = load_image('human_dog\\testA\\200601.jpg').to(device)
image2 = load_image('human_dog\\testB\\flickr_dog_000079.jpg').to(device)

image1_feature = resnet1(image1)
image1_feature = image1_feature.unsqueeze(1).unsqueeze(1).expand(-1,-1,256,256)
image1_with_feature = torch.cat([image1,image1_feature],dim=1)

image2_feature = resnet1(image2)
image2_feature = image2_feature.unsqueeze(1).unsqueeze(1).expand(-1,-1,256,256)
image2_with_feature = torch.cat([image2,image2_feature],dim=1)

output_image = predict(G_AA,image1_with_feature ,image2_with_feature)
# output_image = output_image - output_image.min()
# output_image = output_image / output_image.max()

# output_image = output_image.squeeze()  # 假设输出是图像格式，调整通道
# output_image = output_image.permute(1,2,0)
# output_image=output_image.to('cpu')
# # 步骤 5: 可视化输出图像
# plt.imshow(output_image.numpy())
# plt.title('Output Image')
# plt.show()

In [38]:
print(output_image.shape)

torch.Size([1, 3, 256, 256])
